In [3]:
import ROOT
%jsroot on
# calculate events ratio from vertex histograms
path='/home/prozorov/dev/star/jets_pp_2012/output/'
triggers = ['JP2', 'HT2', 'MB']

bad_runs_JP2=[  13048019, 13048092, 13048093,
                13049006, 13049007, 13051074,
                13051099, 13052061, 13061035,
                13064067, 13068060, 13069023,
                13070061, 13063012]

### plot Vertex Z distributions

In [4]:
import math
# calculate events ratio from vertex histograms
event_ratio = {}

canvas = ROOT.TCanvas('canvas', '', 1200, 400)
canvas.Divide(3, 1)
canvas.Draw()

line = ROOT.TLine()
line.SetLineColor(ROOT.kRed)
line.SetLineStyle(2)

text =ROOT.TLatex()
text.SetTextSize(0.03)
text.SetTextColor(ROOT.kRed)

for i, trigger in enumerate(triggers):
    file = ROOT.TFile(path + f'jets_{trigger}.root')
    vertex_hist = file.Get('hVertexZ')
    vertex_hist.SetName(f'hVertexZ_{trigger}')
    vertex_hist.SetDirectory(0)
    vertex_hist.SetTitle(trigger)
    vertex_hist.GetXaxis().SetRangeUser(-35, 35)

    canvas.cd(i + 1)
    vertex_hist.Draw('hist')
    gaus_fit = ROOT.TF1(f"gaus_{trigger}", "gaus", -35, 35)
    vertex_hist.Fit(gaus_fit, "REMQ")

    binwidth = vertex_hist.GetBinWidth(2)
    integral = vertex_hist.Integral(vertex_hist.FindBin(-30), vertex_hist.FindBin(30)) * binwidth
    total_integral = math.sqrt(2 * math.pi) * gaus_fit.GetParameter(0) * gaus_fit.GetParameter(2)
    ratio = integral / total_integral
    event_ratio[trigger] = ratio

    line.DrawLine(-30, 0, -30, 1.05 * vertex_hist.GetMaximum())
    line.DrawLine(30, 0, 30, 1.05 * vertex_hist.GetMaximum())
    text.DrawLatexNDC(0.2, 0.3, f'{ratio:.2f} total')


Warning in <TClass::Init>: no dictionary for class TStarJetVectorJet is available
Warning in <TClass::Init>: no dictionary for class TStarJetVector is available


### Read raw pt distributions

In [5]:

pt_vs_run_raw = {}
pt_vs_run_trigger = {}
infile_2d = ROOT.TFile("pt_vs_run.root")
for trigger in triggers:
    pt_vs_run_raw[trigger] = infile_2d.Get(f'{trigger}_pt_vs_run')
    pt_vs_run_raw[trigger].SetDirectory(0)
    if trigger in ['JP2', 'HT2']:
        pt_vs_run_trigger[trigger] = infile_2d.Get(f'{trigger}_pt_vs_run_trigger')
        pt_vs_run_trigger[trigger].SetDirectory(0)

pt_vs_run_trigger['MB']= pt_vs_run_raw['MB']


### Get analyzed events histograms

In [6]:
analyzed_events = {}
for trigger in triggers:
    file_name = path + f'jets_{trigger}.root'

    my_file=ROOT.TFile(file_name)
    analyzed_events[trigger]= my_file.Get('hEventsRun')
    analyzed_events[trigger].SetDirectory(0)

### Read all prescales, livetimes, maxnevents, luminosities

In [7]:
# get 1d histograms - presacles, livetime, maxnevents
lumi=ROOT.TFile("lumi.root")
prescale={}
livetime={}
maxnevent={}
luminosity={}
for trigger in triggers:
    prescale[trigger]= lumi.Get(f'prescale_{trigger}')
    livetime[trigger]= lumi.Get(f'livetime_{trigger}')
    maxnevent[trigger]= lumi.Get(f'nevents_{trigger}')
    luminosity[trigger]= lumi.Get(f'luminosity_{trigger}')

print()


### Get Trigger efficiency from embedding

In [8]:
trigger_efficiency={}
trigger_mc_efficiency={}

# load trigger efficiency from embedding
for trigger in triggers:
    input_file = ROOT.TFile(f"./plots/embedding_root/{trigger}_reco.root", "READ")
    trigger_efficiency[trigger]=input_file.Get(f"h_ratio").Clone(f'h_ratio_{trigger}')
    # input_file_mc = ROOT.TFile(f"./plots/embedding_root/{trigger}_mc.root", "READ")
    # trigger_mc_efficiency[trigger]=input_file_mc.Get(f"h_ratio"). Clone(f'h_ratio_mc_{trigger}')
    trigger_efficiency[trigger].SetDirectory(0)
    # trigger_mc_efficiency[trigger].SetDirectory(0)

print("Trigger efficiency loaded")

Trigger efficiency loaded


In [9]:
import math
def divide(num, denom):
    hnew = num.Clone()  # Clone the first histogram
    for j in range(1, hnew.GetNbinsY() + 1):
        for i in range(1, hnew.GetNbinsX() + 1):
            bin_content = denom.GetBinContent(i)
            if bin_content == 0:
                scale = 0
            else:
                scale = 1 / bin_content
            hnew.SetBinContent(i, j, hnew.GetBinContent(i, j) * scale)
            hnew.SetBinError(i, j, hnew.GetBinError(i, j) * scale)
    hnew.SetDirectory(0)  # Detach from any file
    return hnew

def multiply(num, denom):
    hnew = num.Clone()  # Clone the first histogram
    for j in range(1, hnew.GetNbinsY() + 1):
        for i in range(1, hnew.GetNbinsX() + 1):
            scale = denom.GetBinContent(i)
            hnew.SetBinContent(i, j, hnew.GetBinContent(i, j) * scale)
            hnew.SetBinError(i, j, hnew.GetBinError(i, j) * scale)
    hnew.SetDirectory(0)  # Detach from any file
    return hnew

# make 1d histogram out of this histograms
def make_run_projection(hist):
    counter_runs=0
    name = hist.GetName()
    temp = hist.ProjectionY('', 1, 1)
    output=ROOT.TH1D(name+'_per_run', name+ ';'+hist.GetYaxis().GetTitle()+';counts per event', temp.GetNbinsX(), temp.GetXaxis().GetXmin(), temp.GetXaxis().GetXmax())
    for i in range(1, hist.GetNbinsX() + 1):
         projection = hist.ProjectionY(f'{name}_{i}', i, i)
         runid = int(hist.GetXaxis().GetBinLabel(i))
         # check if run is in bad runs
         if runid in bad_runs_JP2:
             continue
         if projection.Integral() == 0:
            continue
         counter_runs+=1
         output.Add(projection)
    output.Scale(1/counter_runs)
    output.SetDirectory(0)  # Detach from any file
    
    return output

# go by bin by bin and multiply by trigger efficiency interpolated
def divide_by_efficiency(data, eff):
    for i in range(1, data.GetNbinsX()+1):
        bin_content = data.GetBinContent(i)
        bin_error = data.GetBinError(i)
        bin_center = data.GetBinCenter(i)
        # get trigger efficiency
        eff_value = eff.GetBinContent(eff.GetXaxis().FindBin(bin_center))
        eff_error = eff.GetBinError(eff.GetXaxis().FindBin(bin_center))
        # check if trigger efficiency is 0
        if eff_value == 0:
            eff_value = 1
        # set new value
        data.SetBinContent(i, bin_content /eff_value)
        # calculate uncertainty propagation
        # check if not 0
        if bin_content == 0:
            continue
        new_bin_error = bin_content / eff_value * math.sqrt((bin_error/bin_content)**2 + (eff_error/eff_value)**2)
        data.SetBinError(i, new_bin_error)
    return data


### Combine all histograms on per-run basis

In [10]:
my_cross_section = {}

for trigger in triggers:
    print(f'Calculating cross section for {trigger}')
    temp = pt_vs_run_trigger[trigger]
    # =============================================
    # normalize by events
    temp = divide(temp, analyzed_events[trigger]) 
    # multiply by prescale
    if trigger != 'MB':
        temp = multiply(temp, prescale[trigger]) 
    # divide by luminosity
    temp = divide(temp, luminosity[trigger]) 
    # multiply by livetime
    temp = multiply(temp, livetime[trigger]) 
    # multiply by maxnevent
    temp = multiply(temp, maxnevent[trigger]) 
    # scale by ratio of passed events
    temp.Scale(1./event_ratio[trigger])
    # ============================================
    # make run projection
    temp = make_run_projection(temp)
    # trigger efficiency
    temp = divide_by_efficiency(temp, trigger_efficiency[trigger])

    # normalize by bin width
    for i in range(1, temp.GetNbinsX()+1):
        bin_width = temp.GetBinWidth(i)
        bin_content = temp.GetBinContent(i)
        bin_error = temp.GetBinError(i)
        temp.SetBinContent(i, bin_content / bin_width)
        temp.SetBinError(i, bin_error / bin_width)

    my_cross_section[trigger] = temp


Calculating cross section for JP2
Calculating cross section for HT2
Calculating cross section for MB


In [11]:
# load Dmitry's histograms
path='./jet_cross_section_dmitriy.root'
file=ROOT.TFile(path)
base_hist=file.Get('crossSection_systematic')

In [12]:
def divide_different_bins (num, denom):
    hnew = num.Clone()  # Clone the first histogram
    for i in range(1, hnew.GetNbinsX() + 1):
        scale = denom.Interpolate(hnew.GetXaxis().GetBinCenter(i))
        # scale =denom.GetBinContent( denom.GetXaxis().FindBin(hnew.GetXaxis().GetBinCenter(i)))
        if scale == 0:
            scale = 1
        hnew.SetBinContent(i, hnew.GetBinContent(i) /scale)
        hnew.SetBinError(i,  hnew.GetBinError(i) / scale)
    hnew.SetDirectory(0)  # Detach from any file
    return hnew

In [13]:
# draw all cross sections

ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetOptTitle(0)
colors={
    'JP2': ROOT.kAzure-1,
    'HT2': ROOT.kRed+1,
    'MB': ROOT.kOrange+1,
    'Dmitriy': ROOT.kViolet,
}

canvas = ROOT.TCanvas('canvas', 'canvas', 1200, 1200)

canvas.Divide(1, 2)
# Upper pad for histograms
pad1 = canvas.cd(1)
pad1.SetPad(0.0, 0.5, 1.0, 1.0)
pad1.SetTopMargin(0.1)
pad1.SetBottomMargin(0.0)
pad1.SetLogy()

canvas.Draw()

canvas.cd(1)
base_hist.GetYaxis().SetRangeUser(1e0+0.2, 1e7)
base_hist.GetXaxis().SetRangeUser(base_hist.GetXaxis().GetXmin()-1, base_hist.GetXaxis().GetXmax())
base_hist.SetMarkerColor(colors['Dmitriy'])
base_hist.SetLineColor(colors['Dmitriy'])
base_hist.SetMarkerStyle(29)
base_hist.SetMarkerSize(2)
base_hist.Draw('E1')

for i, trigger in enumerate(triggers): 
    my_cross_section[trigger].SetLineColor( colors[trigger])
    my_cross_section[trigger].SetMarkerColor( colors[trigger])
    my_cross_section[trigger].SetMarkerStyle(20+i)
    my_cross_section[trigger].Draw('E1 same')

legend = ROOT.TLegend(0.7, 0.5, 0.88, 0.88)
for trigger in triggers:
    legend.AddEntry(my_cross_section[trigger], trigger, "lep")
legend.AddEntry(base_hist, 'Dmitriy', "lep")
legend.Draw()
#========================
# Lower pad for ratio
pad2 = canvas.cd(2)
pad2.SetPad(0.0, 0.0, 1.0, 0.5)
pad2.SetTopMargin(0.0)
pad2.SetBottomMargin(0.3)

canvas.cd(2)

ratio_hist={}

for trigger in ['JP2', 'HT2', 'MB']:
    ratio_hist[trigger] = my_cross_section[trigger].Clone(f"ratio_{trigger}")
    ratio_hist[trigger] = divide_different_bins(ratio_hist[trigger], base_hist)
    ratio_hist[trigger].GetYaxis().SetRangeUser(0, 3.95)
    ratio_hist[trigger].GetXaxis().SetRangeUser(base_hist.GetXaxis().GetXmin()+0.1, base_hist.GetXaxis().GetXmax())
    ratio_hist[trigger].GetYaxis().SetTitle('Ratio to Dmitry')
    ratio_hist[trigger].GetXaxis().SetTitle('p_{t} [GeV/c]')
    ratio_hist[trigger].SetDirectory(0) 
    ratio_hist[trigger].Draw('E1 same')
     # Detach from any file

# draw line at 1
line.SetLineColor(colors['Dmitriy'])
line.DrawLine(base_hist.GetXaxis().GetXmin(), 1, base_hist.GetXaxis().GetXmax(), 1)
line.Draw()


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
